In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#from math import sin, cos, sqrt, atan2, radians
import math as math

In [ ]:
train = pd.read_csv("C:/Users/shabbir.hasan/Documents/Hasan/Anaconda_rep/Kaggle/New_York_City_Taxi_Fare_Prediction/generated_datasets/train_2.csv")

### Changing field type

In [ ]:
# converting data type of columns
train['passenger_count'] = train['passenger_count'].astype('category')
train['year'] = train['year'].astype('category')
train['month'] = train['month'].astype('category')
train['day_of_month'] = train['day_of_month'].astype('category')
train['day_of_week'] = train['day_of_week'].astype('category')
train['weekend_weekday'] = train['weekend_weekday'].astype('category')
train['date'] = pd.to_datetime(train['date'])

###  handling time

In [ ]:
#Day_of_month
train['hour'] = train.time.apply(lambda x: x[0:2])
train['minute'] = train.time.apply(lambda x: x[3:5])
train['time_30_min_bin'] = train.minute.apply(lambda x: '30' if int(x) >= 30 else '0')
train['time_15_min_bin'] = train.minute.apply(lambda x: '0' if int(x) < 15 
                                              else ('15' if int(x) >= 15 and int(x) < 30 
                                                    else ('30' if int(x) >=30 and int(x) < 45 
                                                          else '45')))

In [ ]:
#dropping unused columns
train =  train.drop('minute', 1)
train =  train.drop('time', 1)
train =  train.drop('year_month', 1)

In [ ]:
train.dtypes

In [ ]:
# changing data type
train['time_15_min_bin'] = train['time_15_min_bin'].astype('category')
train['time_30_min_bin'] = train['time_30_min_bin'].astype('category')
train['hour'] = train['hour'].astype('category')

In [ ]:
train.head()

In [ ]:
#checking for null values
train.isnull().values.any()

#### Cleaning lat long data to calculate radial distance

In [ ]:
train['pickup_latitude'] = train['pickup_latitude'].round(6)
train['pickup_longitude'] = train['pickup_longitude'].round(6)
train['dropoff_latitude'] = train['dropoff_latitude'].round(6)
train['dropoff_longitude'] = train['dropoff_longitude'].round(6)

In [ ]:
#removing lat and long with 0.0 values
train = train[train['pickup_latitude'] != 0.0]
train = train[train['pickup_longitude'] != 0.0]
train = train[train['dropoff_latitude'] != 0.0]
train = train[train['dropoff_longitude'] != 0.0]

In [ ]:
#train.to_csv('clean_3.csv', index=False )

In [ ]:
#train = pd.read_csv('clean_3.csv')

#####  considering lat long only between -90 and +90

In [ ]:
train = train[(train["pickup_latitude"] >= -90.0) & (train["pickup_latitude"] <= 90.0)]
train = train[(train["pickup_longitude"] >= -90.0) & (train["pickup_longitude"] <= 90.0)]
train = train[(train["dropoff_latitude"] >= -90.0) & (train["dropoff_latitude"] <= 90.0)]
train = train[(train["dropoff_longitude"] >= -90.0) & (train["dropoff_longitude"] <= 90.0)]

In [ ]:
ax = sns.boxplot(x=train["pickup_latitude"])

In [ ]:
ax = sns.boxplot(x=train["pickup_longitude"])

In [ ]:
ax = sns.boxplot(x=train["dropoff_latitude"])

In [ ]:
ax = sns.boxplot(x=train["dropoff_longitude"])

In [ ]:
#outlier removing
train = train[(train["pickup_latitude"] >= 39) & (train["pickup_latitude"] <= 46)]
train = train[(train["pickup_longitude"] <= -70) & (train["pickup_longitude"] >= -76)]
train = train[(train["dropoff_latitude"] >= 39) & (train["dropoff_latitude"] <= 46)]
train = train[(train["dropoff_longitude"] <= -70) & (train["dropoff_longitude"] >= -76)]

In [ ]:
%%time
sns.distplot(train["pickup_longitude"])

In [ ]:
train.shape

#### calculating radial distance

In [ ]:
#fun to calc dist
def radialDistance(lat1 , lon1, lat2, lon2):
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return round(d,2)

In [ ]:
%%time
train['distance_travelled'] = train.apply(lambda x: radialDistance(x['pickup_latitude'], x['pickup_longitude'], 
                                           x['dropoff_latitude'], x['dropoff_longitude']), axis=1)

In [ ]:
#train = train.drop('date', axis=1)

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
train.to_csv('clean_v2_01-02_v2.csv', index=False )